In [1]:
import pandas as pd
import os
os.chdir("/home/yangkai/codegen-detection/")
hum_data = pd.read_pickle("dataset/mbpp_hum_test.pkl")
gen_data = pd.read_pickle("dataset/mbpp_gen_test.pkl")

In [51]:
import json
def load_json_dataset(pth):
    dataset = []
    cnt = 0
    with open(pth,"r") as f:
        for line in f:
            try:
                dataset.append(json.loads(line.strip()))
                cnt += 1
            except:
                dataset.append([{},{}])

    return dataset


idx = 8

hum_expl = load_json_dataset(f'prompt_gpt/mbpp_hum_test_vicuna13b_rewrite{idx}.jsonl')
gen_expl = load_json_dataset(f'prompt_gpt/mbpp_gen_test_vicuna13b_rewrite{idx}.jsonl')
len(gen_expl)

233

In [52]:
import re
def extract_starcoder_code(resp):
    try:
        text = resp[1][0]['content']
    except KeyError:
        return ''
    
    pattern = r'```\n*python\n(.+?)\n```'
    code_blocks = re.findall(pattern, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]
    
    pattern2 = r"```\n*(.+?)\n*```"
    code_blocks = re.findall(pattern2, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]
    
    pattern3 = r"\n+(def|import|from)(.+)"
    code_blocks = re.findall(pattern3, text, re.DOTALL)
    if len(code_blocks) > 0:
        # print(code_blocks[0][0] + code_blocks[0][1])
        # print("=============")
        text = code_blocks[0][0] + code_blocks[0][1]
        code_pattern = r"(.+)\n+(This|In this|Here's).*"
        blocks = re.findall(code_pattern,text, re.DOTALL)
        if len(blocks) > 0:
            # print(blocks[0][0]+"\n=========")
            return blocks[0][0]
        else:
            return text

    return ""


import re
def extract_gpt_code(resp):
    try:
        text = resp['choices'][0]['message']['content']
        # text = resp[1][0]['generated_text']
    except KeyError:
        return ''
    pattern = r'```python\n(.+?)\n```'
    code_blocks = re.findall(pattern, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][0]['finish_reason'] == 'length':
        pattern2 = r'```python\n(.+?)'
    else:
        pattern2 = r'``` python\n(.+?)\n```'

    # pattern2 = r'```python\n(.+?)'
    code_blocks = re.findall(pattern2, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    pattern3 = r"```\n(.+?)\n```"
    code_blocks = re.findall(pattern3, text, re.DOTALL)
    if len(code_blocks) > 0:
        return code_blocks[0]

    if resp['choices'][0]['finish_reason'] == 'length':
        pattern4 = r"```\n(.+?)"
        code_blocks = re.findall(pattern4, text, re.DOTALL)
        if len(code_blocks) > 0:
            return code_blocks[0]





failed_cases = []
for resp in hum_expl:
    func = extract_starcoder_code(resp)
    if len(func) == 0 and len(resp[1]) > 0:
        failed_cases.append(resp[1][0]['content'])

    # if case == 0:
    #     case_1.append(func)
    # elif case == 1:
    #     case_2.append(func)
    # elif case == 2:
    #     case_3.append(func)
    

len(failed_cases)

1

In [53]:
key_name = f'vicuna13b_resample_code{idx}'
hum_data[key_name] = hum_expl
hum_data[key_name] = hum_data[key_name].apply(extract_starcoder_code)

gen_data[key_name] = gen_expl
gen_data[key_name] = gen_data[key_name].apply(extract_starcoder_code)

In [54]:
from prepro.code_prepro import CommentsRemover,PygmentsTokenizer,EmptyLinesRemover
tokenizer = PygmentsTokenizer('Python')
comment_remover = CommentsRemover(tokenizer)
emptyline_remover = EmptyLinesRemover()

In [55]:
cleaned_name = f'vicuna13b_resample_code_cleaned{idx}'
temp = gen_data[key_name].apply(comment_remover.filter)
gen_data[cleaned_name] = temp.apply(emptyline_remover.filter)

temp = hum_data[key_name].apply(comment_remover.filter)
hum_data[cleaned_name] = temp.apply(emptyline_remover.filter)

In [56]:
hum_data[cleaned_name]

6      def check_min_heap(arr, i):\n    if i >= len(a...
7      def pass_validity(p):\n    x = True\n    while...
8      def multiply_int(x, y):\n    return x * y if y...
13     def solve(a, n):\n    mx = max(a[:-1]) if a el...
14     def get_odd_occurence(arr, arr_size):\n    cou...
                             ...                        
952    def sort_tuple(tup):\n    n = len(tup)\n    fo...
957    def is_sublist(l, s):\n    sub_set = False\n  ...
960    def first(arr, x, n):\n    low = 0\n    high =...
961    def find_max(arr, low, high):\n    if high < l...
970    import re\nregex = r'^[a-z]$|^([a-z]).*\1$'\nd...
Name: vicuna13b_resample_code_cleaned8, Length: 233, dtype: object

In [58]:
gen_data.to_pickle("dataset/mbpp_gen_test.pkl")
hum_data.to_pickle("dataset/mbpp_hum_test.pkl")